In [1]:
from skfem import *
import numpy as np
from utils import solver_iter_krylov, solver_iter_pyamg, solver_iter_mgcg
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time

# aaa

In [2]:
for ii in range(1):
    pi = np.pi
    sin = np.sin
    cos = np.cos
    exp = np.exp

    # parameters

    tol = 1e-8
    intorder = 5
    refine_time = 5
    epsilon_range = 1
    element_type = 'P1'
    sigma = 5
    penalty = False
    example = 'ex1'

    # end of parameters

    save_path = 'log/' + example + '_' + element_type + '_' + (
        'pen' if penalty else 'nopen') + '_' + '{}'.format(
            datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

    # # output to txt
    # class Logger(object):
    #     def __init__(self, filename=save_path+'.txt', stream=sys.stdout):
    # 	    self.terminal = stream
    # 	    self.log = open(filename, 'a')

    #     def write(self, message):
    # 	    self.terminal.write(message)
    # 	    self.log.write(message)

    #     def flush(self):
    # 	    pass

    # sys.stdout = Logger(save_path+'.txt', sys.stdout)

    # print parameters

    print('=======Arguments=======')
    print('example:\t{}'.format(example))
    print('penalty:\t{}'.format(penalty))
    print('element_type:\t{}'.format(element_type))
    print('tol:\t{}'.format(tol))
    print('intorder:\t{}'.format(intorder))
    print('refine_time:\t{}'.format(refine_time))
    print('epsilon_range:\t{}'.format(epsilon_range))
    print('sigma:\t{}'.format(sigma))
    print('save_path:\t{}'.format(save_path))
    print('=======Results=======')

    # functions


    def easy_boundary_penalty(basis):
        '''
        Input basis
        ----------------
        Return D for boundary conditions
        '''

        dofs = basis.find_dofs({
            'left': m.facets_satisfying(lambda x: x[0] == 0),
            'right': m.facets_satisfying(lambda x: x[0] == 1),
            'top': m.facets_satisfying(lambda x: x[1] == 1),
            'buttom': m.facets_satisfying(lambda x: x[1] == 0)
        })

        D = np.concatenate((dofs['left'].nodal['u'], dofs['right'].nodal['u'],
                            dofs['top'].nodal['u'], dofs['buttom'].nodal['u']))
        return D


    def easy_boundary(basis):
        '''
        Input basis
        ----------------
        Return D for boundary conditions
        '''

        dofs = basis.find_dofs({
            'left': m.facets_satisfying(lambda x: x[0] == 0),
            'right': m.facets_satisfying(lambda x: x[0] == 1),
            'top': m.facets_satisfying(lambda x: x[1] == 1),
            'buttom': m.facets_satisfying(lambda x: x[1] == 0)
        })

        D = np.concatenate((dofs['left'].nodal['u'], dofs['right'].nodal['u'],
                            dofs['top'].nodal['u'], dofs['buttom'].nodal['u'],
                            dofs['left'].facet['u_n'], dofs['right'].facet['u_n'],
                            dofs['top'].facet['u_n'], dofs['buttom'].facet['u_n']))
        return D


    @BilinearForm
    def a_load(u, v, w):
        '''
        for $a_{h}$
        '''
        return ddot(dd(u), dd(v))


    @BilinearForm
    def b_load(u, v, w):
        '''
        for $b_{h}$
        '''
        return dot(grad(u), grad(v))


    @BilinearForm
    def wv_load(u, v, w):
        '''
        for $(\nabla \chi_{h}, \nabla_{h} v_{h})$
        '''
        return dot(grad(u), grad(v))


    @BilinearForm
    def penalty_1(u, v, w):
        return ddot(-dd(u), prod(w.n, w.n)) * dot(grad(v), w.n)


    @BilinearForm
    def penalty_2(u, v, w):
        return ddot(-dd(v), prod(w.n, w.n)) * dot(grad(u), w.n)


    @BilinearForm
    def penalty_3(u, v, w):
        return (sigma / w.h) * dot(grad(u), w.n) * dot(grad(v), w.n)


    @BilinearForm
    def laplace(u, v, w):
        '''
        for $(\nabla w_{h}, \nabla \chi_{h})$
        '''
        return dot(grad(u), grad(v))


    @Functional
    def L2uError(w):
        x, y = w.x
        return (w.w - exact_u(x, y))**2


    def get_DuError(basis, u):
        duh = basis.interpolate(u).grad
        x = basis.global_coordinates().value
        dx = basis.dx  # quadrature weights
        dux, duy = dexact_u(x[0], x[1])
        return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))


    def get_D2uError(basis, u):
        dduh = basis.interpolate(u).hess
        x = basis.global_coordinates(
        ).value  # coordinates of quadrature points [x, y]
        dx = basis.dx  # quadrature weights
        duxx, duxy, duyx, duyy = ddexact(x[0], x[1])
        return np.sqrt(
            np.sum(((dduh[0][0] - duxx)**2 + (dduh[0][1] - duxy)**2 +
                    (dduh[1][1] - duyy)**2 + (dduh[1][0] - duyx)**2) * dx))


    def solve_problem1(m, element_type='P1'):
        '''
        old solver for problem 1, can't read f 
        '''
        global K1
        if element_type == 'P1':
            element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
        elif element_type == 'P2':
            element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
        else:
            raise Exception("The element not supported")

        basis = {
            variable: InteriorBasis(m, e, intorder=intorder)
            for variable, e in element.items()
        }  # intorder: integration order for quadrature

        K1 = asm(laplace, basis['w'])
        f1 = asm(f_load, basis['w'])

        # wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
        wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()),
                   solver=solver_iter_krylov(Precondition=True, tol=tol))

        K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
        f2 = asm(wv_load, basis['w'], basis['u']) * wh
        uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])),
                    solver=solver_iter_krylov(Precondition=True, tol=tol))  # cg
        return uh0, basis


    def solve_problem2(m, element_type='P1'):
        '''
        old solver
        '''
        if element_type == 'P1':
            element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
        elif element_type == 'P2':
            element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
        else:
            raise Exception("The element not supported")

        basis = {
            variable: InteriorBasis(m, e, intorder=intorder)
            for variable, e in element.items()
        }

        K1 = asm(laplace, basis['w'])
        f1 = asm(f_load, basis['w'])

        wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()),
                   solver=solver_iter_krylov(Precondition=True, tol=tol))

        fbasis = FacetBasis(m, element['u'])

        p1 = asm(penalty_1, fbasis)
        p2 = asm(penalty_2, fbasis)
        p3 = asm(penalty_3, fbasis)
        P = p1 + p2 + p3

        K2 = epsilon**2 * asm(a_load, basis['u']) + \
            epsilon**2 * P + asm(b_load, basis['u'])
        f2 = asm(wv_load, basis['w'], basis['u']) * wh
        uh0 = solve(*condense(K2, f2, D=easy_boundary_penalty(basis['u'])),
                    solver=solver_iter_krylov(Precondition=True, tol=tol))
        return uh0, basis


    if example == 'ex1':

        @LinearForm
        def f_load(v, w):
            '''
            for $(f, x_{h})$
            '''
            pix = pi * w.x[0]
            piy = pi * w.x[1]
            lu = 2 * (pi)**2 * (cos(2 * pix) * ((sin(piy))**2) + cos(2 * piy) *
                                ((sin(pix))**2))
            llu = -8 * (pi)**4 * (cos(2 * pix) * sin(piy)**2 + cos(2 * piy) *
                                  sin(pix)**2 - cos(2 * pix) * cos(2 * piy))
            return (epsilon**2 * llu - lu) * v

        def exact_u(x, y):
            return (sin(pi * x) * sin(pi * y))**2

        def dexact_u(x, y):
            dux = 2 * pi * cos(pi * x) * sin(pi * x) * sin(pi * y)**2
            duy = 2 * pi * cos(pi * y) * sin(pi * x)**2 * sin(pi * y)
            return dux, duy

        def ddexact(x, y):
            duxx = 2 * pi**2 * cos(pi * x)**2 * sin(pi * y)**2 - 2 * pi**2 * sin(
                pi * x)**2 * sin(pi * y)**2
            duxy = 2 * pi * cos(pi * x) * sin(pi * x) * 2 * pi * cos(pi * y) * sin(
                pi * y)
            duyx = duxy
            duyy = 2 * pi**2 * cos(pi * y)**2 * sin(pi * x)**2 - 2 * pi**2 * sin(
                pi * y)**2 * sin(pi * x)**2
            return duxx, duxy, duyx, duyy

    elif example == 'ex2':

        @LinearForm
        def f_load(v, w):
            '''
            for $(f, x_{h})$
            '''
            x = w.x[0]
            y = w.x[1]
            return ((sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) *
                    (12 * y + ep *
                     ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                       (exp(-1 / ep) + 2 * ep *
                                        (exp(-1 / ep) - 1) + 1))) / ep**2 +
                      (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                            (exp(-1 / ep) + 2 * ep *
                                             (exp(-1 / ep) - 1) + 1))) / ep**2)) -
                    ((pi**2 * sin(pi * x)) / 2 +
                     (ep * pi * (exp(-x / ep) / ep**2 + exp(
                         (x - 1) / ep) / ep**2)) / (2 * (exp(-1 / ep) - 1))) *
                    (ep * (exp(
                        (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                           (3 / (exp(-1 / ep) - 1) + 1 /
                            (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) -
                           (3 * exp(-1 / ep) + 3) / (exp(-1 / ep) - 1) -
                           ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                           (exp(-1 / ep) + 2 * ep *
                            (exp(-1 / ep) - 1) + 1)) + 2 * y *
                     (y**2 - 1)) - ep**2 *
                    (((pi**4 * sin(pi * x)) / 2 -
                      (ep * pi * (exp(-x / ep) / ep**4 + exp(
                          (x - 1) / ep) / ep**4)) / (2 * (exp(-1 / ep) - 1))) *
                     (ep * (exp((y - 1) / ep) *
                            (3 / (exp(-1 / ep) - 1) - 1 /
                             (exp(-1 / ep) + 2 * ep *
                              (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                            (3 / (exp(-1 / ep) - 1) + 1 /
                             (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) -
                            (3 * exp(-1 / ep) + 3) / (exp(-1 / ep) - 1) -
                            ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                            (exp(-1 / ep) + 2 * ep *
                             (exp(-1 / ep) - 1) + 1)) + 2 * y * (y**2 - 1)) - 2 *
                     (12 * y + ep *
                      ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                        (exp(-1 / ep) + 2 * ep *
                                         (exp(-1 / ep) - 1) + 1))) / ep**2 +
                       (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                             (exp(-1 / ep) + 2 * ep *
                                              (exp(-1 / ep) - 1) + 1))) / ep**2)) *
                     ((pi**2 * sin(pi * x)) / 2 +
                      (ep * pi * (exp(-x / ep) / ep**2 + exp(
                          (x - 1) / ep) / ep**2)) / (2 *
                                                     (exp(-1 / ep) - 1))) + ep *
                     (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                         (x - 1) / ep) - exp(-1 / ep) - 1)) /
                      (2 * (exp(-1 / ep) - 1))) *
                     ((exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                       (exp(-1 / ep) + 2 * ep *
                                        (exp(-1 / ep) - 1) + 1))) / ep**4 +
                      (exp((y - 1) / ep) *
                       (3 / (exp(-1 / ep) - 1) - 1 /
                        (exp(-1 / ep) + 2 * ep *
                         (exp(-1 / ep) - 1) + 1))) / ep**4))) * v

        def exact_u(x, y):
            return -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) * (
                    ep * (exp(
                        (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                          (3 / (exp(-1 / ep) - 1) + 1 /
                           (exp(-1 / ep) + 2 * ep *
                            (exp(-1 / ep) - 1) + 1)) - (3 * exp(-1 / ep) + 3) /
                          (exp(-1 / ep) - 1) - ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                          (exp(-1 / ep) + 2 * ep *
                           (exp(-1 / ep) - 1) + 1)) + 2 * y * (y**2 - 1))

        def dexact_u(x, y):
            dux = -(
                (pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
                    (x - 1) / ep) / ep)) /
                (2 *
                 (exp(-1 / ep) - 1))) * (ep * (exp(
                     (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                      (exp(-1 / ep) + 2 * ep *
                                       (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                               (3 / (exp(-1 / ep) - 1) + 1 /
                                                (exp(-1 / ep) + 2 * ep *
                                                 (exp(-1 / ep) - 1) + 1)) -
                                               (3 * exp(-1 / ep) + 3) /
                                               (exp(-1 / ep) - 1) -
                                               ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                         (y**2 - 1))
            duy = (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) /
                   (2 * (exp(-1 / ep) - 1))) * (ep * (
                       (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                                   (exp(-1 / ep) - 1) + 1) +
                       (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                        (exp(-1 / ep) + 2 * ep *
                                         (exp(-1 / ep) - 1) + 1))) / ep -
                       (exp((y - 1) / ep) *
                        (3 / (exp(-1 / ep) - 1) - 1 /
                         (exp(-1 / ep) + 2 * ep *
                          (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
            return dux, duy

        def ddexact(x, y):
            duxx = (
                (pi**2 * sin(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep**2 + exp(
                    (x - 1) / ep) / ep**2)) /
                (2 *
                 (exp(-1 / ep) - 1))) * (ep * (exp(
                     (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                      (exp(-1 / ep) + 2 * ep *
                                       (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                               (3 / (exp(-1 / ep) - 1) + 1 /
                                                (exp(-1 / ep) + 2 * ep *
                                                 (exp(-1 / ep) - 1) + 1)) -
                                               (3 * exp(-1 / ep) + 3) /
                                               (exp(-1 / ep) - 1) -
                                               ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                         (y**2 - 1))
            duxy = ((pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
                (x - 1) / ep) / ep)) / (2 * (exp(-1 / ep) - 1))) * (ep * (
                    (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1) +
                    (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                     (exp(-1 / ep) + 2 * ep *
                                      (exp(-1 / ep) - 1) + 1))) / ep -
                    (exp((y - 1) / ep) *
                     (3 / (exp(-1 / ep) - 1) - 1 /
                      (exp(-1 / ep) + 2 * ep *
                       (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
            duyx = duxy
            duyy = -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
                (x - 1) / ep) - exp(-1 / ep) - 1)) /
                     (2 * (exp(-1 / ep) - 1))) * (12 * y + ep * (
                         (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                          (exp(-1 / ep) + 2 * ep *
                                           (exp(-1 / ep) - 1) + 1))) / ep**2 +
                         (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                               (exp(-1 / ep) + 2 * ep *
                                                (exp(-1 / ep) - 1) + 1))) / ep**2))
            return duxx, duxy, duyx, duyy

    elif example == 'ex3':

        @LinearForm
        def f_load(v, w):
            pix = pi * w.x[0]
            piy = pi * w.x[1]
            return (2 * pi**2 * sin(pix) * sin(piy)) * v

        def exact_u(x, y):
            return sin(pi * x) * sin(pi * y)

        def dexact_u(x, y):
            dux = pi * cos(pi * x) * sin(pi * y)
            duy = pi * cos(pi * y) * sin(pi * x)
            return dux, duy

        def ddexact(x, y):
            duxx = -pi**2 * sin(pi * x) * sin(pi * y)
            duxy = pi * cos(pi * x) * pi * cos(pi * y)
            duyx = duxy
            duyy = -pi**2 * sin(pi * y) * sin(pi * x)
            return duxx, duxy, duyx, duyy

    else:
        raise Exception('Example not supported')

=======Arguments=======
example:	ex1
penalty:	False
element_type:	P1
tol:	1e-08
intorder:	5
refine_time:	5
epsilon_range:	1
sigma:	5
save_path:	log/ex1_P1_nopen_2020-10-25_14-47-36
=======Results=======


# bbb

In [3]:
def solve_problem1(m, element_type='P1'):
    '''
    old solver for problem 1, can't read f 
    '''
    global K1
    if element_type == 'P1':
        element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
    elif element_type == 'P2':
        element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
    else:
        raise Exception("The element not supported")

    basis = {
        variable: InteriorBasis(m, e, intorder=intorder)
        for variable, e in element.items()
    }  # intorder: integration order for quadrature

    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])

    # wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
    wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_pyamg(tol=tol))

    K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
    f2 = asm(wv_load, basis['w'], basis['u']) * wh
    uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])),
                solver=solver_iter_krylov(Precondition=True, tol=tol))  # cg
    return uh0, basis

time_start = time.time()

df_list = []
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()

        if penalty:
            uh0, basis = solve_problem2(m, element_type)
        else:
            uh0, basis = solve_problem1(m, element_type)

        U = basis['u'].interpolate(uh0).value

        # compute errors

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    # store data
    data = np.array([L2s, H1s, H2s, epus])
    df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
    df_list.append(df)

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))

time_end = time.time()

epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.83  0.87  0.71  0.70
2^-3  2.19  1.76  1.02  0.98
2^-4  2.16  1.93  1.05  1.02
2^-5  2.06  1.98  1.02  1.01


In [4]:
tol = 1e-10

In [5]:
m=MeshTri()
m.refine(6)
print((2**7)**2)

16384


In [6]:
if element_type == 'P1':
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
elif element_type == 'P2':
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
else:
    raise Exception("The element not supported")

In [7]:
%%time
basis = {
    variable: InteriorBasis(m, e, intorder=intorder)
    for variable, e in element.items()
}  # intorder: integration order for quadrature

Wall time: 433 ms


In [8]:
epsilon = 1
ep = epsilon

In [9]:
%%time
K1 = asm(laplace, basis['w'])
f1 = asm(f_load, basis['w'])

Wall time: 36.9 ms


# First part

- MGCG

In [10]:
%%time
# wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol))
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_mgcg(tol=tol, verbose=True))

1221.2998176163405
1228.681754125302
1228.853184454973
1228.8564301772483
1228.8563344021354
1228.8563301706492
1228.8563301528486
1228.85633015484
cg converged to tol=1e-10 and atol=default
Wall time: 25.9 ms


- PCG

In [11]:
%%time
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_krylov(Precondition=True, tol=tol, verbose=True))
# wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_pyamg(tol=tol))

156.629768337169
255.4082948141724
322.8516400983561
364.3453815257323
398.3808573978177
429.74436923223516
455.4713178021487
481.21185665350004
503.35129250475126
526.2504441067845
546.8950468089681
568.6549901806193
589.0412865427311
610.7739172176107
631.7275721609968
654.1896755072682
676.2787436614187
699.9751359389608
723.563095627376
748.7834878798088
774.036385741646
800.8529250310124
827.715355209077
855.9477731693275
884.0902253727648
913.244257019731
941.9943014592124
971.2006051794165
999.5021176542969
1027.5201593227494
1053.9990940303492
1079.368665801826
1102.5724361195448
1123.9404336821574
1142.7180362601523
1159.232545544419
1173.0821517222482
1184.6405480283584
1193.820359699484
1201.0274724427416
1206.3955036634693
1210.3631688402445
1213.2909497352757
1215.7951122519319
1218.5327207962273
1221.9987773045357
1225.0151841142583
1226.3889265364035
1226.8878508339976
1227.2785503617267
1227.7553636358703
1228.0562253806102
1228.2166052555951
1228.3896293882876
1228.517

- AMG

In [12]:
%%time
wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_pyamg(tol=tol, verbose=True))

1111.5486792763174
1217.3000164143227
1227.6948676388201
1228.7377609917787
1228.8440715805762
1228.8550493328862
1228.8561951173826
1228.8563158047193
1228.8563286190245
1228.8563299894133
1228.8563301369293
Wall time: 27.9 ms


# Assemble

In [13]:
%%time
K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
f2 = asm(wv_load, basis['w'], basis['u']) * wh

Wall time: 114 ms


- MGCG

In [18]:
tol = 1e-10

In [19]:
%%time
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_mgcg(tol=tol, verbose=True))
# uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_pyamg(tol=tol)) 

10.378909285035077
49.514401437129855
64.1195739921758
79.76101258865098
97.47563025878125
111.6121518923994
122.19019851741268
130.7621555410077
137.19521689592392
141.78259720719356
145.1410105363027
147.486249864447
149.08158666366975
150.18199519789547
150.93668560745706
151.45169652001428
151.77928565653536
151.99148260715194
152.1414508572858
152.2380397722096
152.30470069219808
152.34967742573457
152.38107218025428
152.41080825489138
152.43871312821605
152.46927154101562
152.49106895815157
152.49827435262029
152.50374313081988
152.50536353621442
152.50615986770032
152.5074423191135
152.5077223364433
152.50860168490462
152.50936156279573
152.50969717413486
152.51028757840797
152.5105072702787
152.51068914642644
152.51092892713362
152.5110014369799
152.51116790568022
152.5112900221372
152.51129649966612
152.51130864239377
152.51121700742135
152.5111179167432
152.5110469665409
152.51093677833896
152.51089283371797
152.5108791334323
152.51086946835704
152.5108739432524
152.510864653

- PCG

In [20]:
%%time
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_krylov(Precondition=True, tol=tol, verbose=True))  
# uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_pyamg(tol=tol))

0.24526766612746298
0.3526877400283238
0.38727966707702705
1.2394582468416848
1.6789803553670908
2.1573525528148854
2.6075040313770597
3.2952290343778348
3.8429071102570047
4.284111165699713
4.795006785115825
5.144758555771061
5.93000750856852
6.362446213131537
6.965636397757222
7.51363374298579
8.018434365232372
8.53209655838268
9.018597257747963
9.483052043739185
10.048330091808577
10.655483037174632
11.146012747516181
11.709381572642055
12.21686262283367
12.730145693867728
13.191431643584494
13.693529868183024
14.182139900004954
14.792763728786243
15.280300305470378
15.789463970894737
16.290506983117684
16.80905963533788
17.229988276565972
17.735335210904047
18.223457895930398
18.755207524874653
19.242613816835227
19.733324492611295
20.200180027991347
20.694270551666794
21.122610890978088
21.581963398731094
22.07018887486312
22.56367642503746
23.021307722133887
23.50233431399592
23.949354293858143
24.395082428214838
24.831873988126926
25.2712849537693
25.71730870962574
26.1894387479

152.51543232811738
152.51508832235555
152.5147711215997
152.51445310237952
152.5141330530054
152.51381480201016
152.51356768029993
152.51337565003254
152.5131987798024
152.51303810389268
152.51288592685336
152.51272322248911
152.5125734949544
152.5124220277417
152.51226102906284
152.512126878516
152.51201454461304
152.51190971043073
152.51180970556717
152.51171735593792
152.51162749371193
152.51153975760377
152.51145574813424
152.51137857156863
152.51131903088984
152.51126890342988
152.51122218509647
152.51118385582325
152.51115224542895
152.51112154395872
152.5110912769379
152.51106090754996
152.51103579398227
152.5110140544296
152.5109950302941
152.51097873562696
152.51096444913784
152.51095049709826
152.5109371638746
152.510923909908
152.51091164928604
152.51090118042623
152.51089107271088
152.51088121579602
152.51087253407962
152.51086341543288
152.5108548812152
152.51084751088334
152.5108415949212
152.5108375215454
152.5108348493458
152.5108328238495
152.51083130231515
152.5108299

- AMG

In [17]:
%%time 
uh0 = solve(*condense(K2, f2, D=easy_boundary(basis['u'])), solver=solver_iter_pyamg(tol=tol, verbose=True)) 

1.0018568537887995
1.8476164502882177
2.67049497210122
3.4755228218283984
4.264471382980285
5.038646942136796
5.799153236757333
6.546946476130318
7.282865087127398
8.007651093048711
8.721966249627009
9.426404499213643
10.12150178384961
10.807743927248772
11.485573073807792
12.155393025695917
12.81757372139113
13.472455033206087
14.120350016117587
14.761547708462466
15.396315562301925
16.024901564610445
16.647536098038895
17.264433580603498
17.875793916415365
18.4818037839178
19.082637783635402
19.67845946386945
20.269422239890552
20.85567021983004
21.437338948539253
22.014556079083594
22.587441980206222
23.156110286972467
23.72066840086212
24.2812179447735
24.83785517772118
25.390671373419142
25.939753166442475
26.485182869224072
27.027038762764786
27.565395363613135
28.100323669382174
28.631891384824378
29.16016313026598
29.685200634013977
30.20706291017692
30.725806423194143
31.241485240236223
31.754151172524672
32.26385390651276
32.77064112577911
33.27455862440638
33.77565041253798


128.1059547602475
128.20389678100892
128.3014461757703
128.3986045155236
128.49537336498793
128.59175428263327
128.68774882070912
128.7833585252651
128.87858493617796
128.97342958717593
129.06789400586462
129.16197971375058
129.25568822626406
129.34902105278633
129.44197969667326
129.53456565527668
129.62678041997216
129.71862547618053
129.8101023033927
129.90121237519352
129.9919571592847
130.08233811750935
130.1723567058736
130.2620143745717
130.35131256800938
130.4402527248253
130.528836277916
130.61706465445766
130.70493927592895
130.79246155813433
130.8796329112262
130.9664547397286
131.05292844255874
131.1390554130488
131.22483703896933
131.31027470255052
131.39536978050617
131.48012364405375
131.5645376589366
131.64861318544712
131.73235157844613
131.81575418738768
131.8988223563375
131.98155742399655
132.06396072372223
132.14603358354907
132.22777732620986
132.3091932691565
132.39028272458222
132.4710469994424
132.55148739547414
132.63160520921878
132.7114017320412
132.79087825

148.50375013693503
148.51984779725018
148.53588080234928
148.5518494118063
148.5677538841525
148.58359447688287
148.59937144645986
148.61508504831698
148.63073553686198
148.64632316548395
148.6618481865531
148.67731085142898
148.6927114104622
148.70805011299944
148.72332720738632
148.7385429409726
148.75369756011642
148.76879131018626
148.7838244355666
148.79879717966182
148.81370978490034
148.82856249273686
148.8433555436583
148.85808917718606
148.87276363188084
148.8873791453471
148.90193595423435
148.91643429424346
148.93087440013022
148.94525650570776
148.95958084385182
148.97384764650323
148.98805714467235
149.0022095684432
149.01630514697624
149.03034410851237
149.0443266803768
149.05825308898343
149.0721235598364
149.08593831753515
149.0996975857801
149.11340158737147
149.12705054421627
149.14064467733246
149.15418420684975
149.1676693520148
149.18110033119527
149.19447736188152
149.20780066069221
149.22107044337614
149.2342869248172
149.2474503190367
149.26056083919713
149.2736